In [ ]:
import os

from app.dataset.dataset import Dataset
from app.dataset.utils.dataset_splitter import SignerDatasetSplitter
from app.features.plotter.frames_plotter import plot_frames
from app.model.model_statistics import ModelStatistics
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

import numpy as np
dataset = Dataset("data/WLASL_v0.3.json")
n_videos = 2000

In [ ]:
videos = dataset.videos
len(videos)

In [ ]:
signers = [video.signer_id for video in videos]
len(signers)

In [ ]:
splitter = SignerDatasetSplitter(videos[:n_videos], frames_split=0.01, extract_features=False)
train_frames, val_frames, test_frames = splitter.train_test_split(test_size=0.2, val_size=0.2, random_state=42)
augmented_train_frames = splitter.apply_data_augmentation(train_frames, num_augmentations=0)

In [ ]:
print(len(train_frames))
print(len(augmented_train_frames))
print(len(val_frames))
print(len(test_frames))

In [ ]:
plot_frames([signer_frame.frame for signer_frame in augmented_train_frames[:30]])

In [ ]:
X_train = [signer_frame.frame for signer_frame in augmented_train_frames]
y_train = [signer_frame.signer_id for signer_frame in augmented_train_frames]
X_val = [signer_frame.frame for signer_frame in val_frames]
y_val = [signer_frame.signer_id for signer_frame in val_frames]
X_test = [signer_frame.frame for signer_frame in test_frames]
y_test = [signer_frame.signer_id for signer_frame in test_frames]

In [ ]:
# Get the total of different labels
num_classes = len(set(y_train) | set(y_val) | set(y_test))
print(num_classes)

In [ ]:
X_train = np.array(X_train)/255.0
X_val = np.array(X_val)/255.0
X_test = np.array(X_test)/255.0

In [ ]:
print('Range originale: [',np.min(X_train),';',np.max(X_train),']')

In [ ]:
X_train = (X_train*2)-1
X_val = (X_val*2)-1
X_test = (X_test*2)-1

In [ ]:
print('Range rimappato: [',np.min(X_train),';',np.max(X_train),']')

In [ ]:
X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
X_test = X_test.astype('float32')

In [ ]:
print(X_train.shape)
print(len(y_train))
print(len(X_val))
print(len(y_val))
print(len(X_test))
print(len(y_test))

In [ ]:
print(X_train[0].shape)

In [ ]:
print(X_train.dtype)

In [ ]:
# Assuming y_train contains person IDs
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)  # Convert IDs to class indices
y_val_encoded = label_encoder.fit_transform(y_val)
y_test_encoded = label_encoder.fit_transform(y_test)

In [ ]:
y_train_onehot = to_categorical(y_train_encoded, num_classes=num_classes)
y_val_onehot = to_categorical(y_val_encoded, num_classes=num_classes)
y_test_onehot = to_categorical(y_test_encoded, num_classes=num_classes)

In [ ]:
def build_vggface():
  model=keras.Sequential(
          [
              layers.Input(shape=(224,224,3),name='input'),
              layers.Conv2D(filters=64, kernel_size=3,padding='same', activation='relu',name='conv1_1-relu1_1'),
              layers.Conv2D(filters=64, kernel_size=3,padding='same', activation='relu',name='conv1_2-relu1_2'),
              layers.MaxPooling2D(pool_size=2, strides=2,name='pool1'),
              layers.Conv2D(filters=128, kernel_size=3,padding='same', activation='relu',name='conv2_1-relu2_1'),
              layers.Conv2D(filters=128, kernel_size=3,padding='same', activation='relu',name='conv2_2-relu2_2'),
              layers.MaxPooling2D(pool_size=2, strides=2,name='pool2'),
              layers.Conv2D(filters=256, kernel_size=3,padding='same', activation='relu',name='conv3_1-relu3_1'),
              layers.Conv2D(filters=256, kernel_size=3,padding='same', activation='relu',name='conv3_2-relu3_2'),
              layers.Conv2D(filters=256, kernel_size=3,padding='same', activation='relu',name='conv3_3-relu3_3'),
              layers.MaxPooling2D(pool_size=2, strides=2,name='pool3'),
              layers.Conv2D(filters=512, kernel_size=3,padding='same', activation='relu',name='conv4_1-relu4_1'),
              layers.Conv2D(filters=512, kernel_size=3,padding='same', activation='relu',name='conv4_2-relu4_2'),
              layers.Conv2D(filters=512, kernel_size=3,padding='same', activation='relu',name='conv4_3-relu4_3'),
              layers.MaxPooling2D(pool_size=2, strides=2,name='pool4'),
              layers.Conv2D(filters=512, kernel_size=3,padding='same', activation='relu',name='conv5_1-relu5_1'),
              layers.Conv2D(filters=512, kernel_size=3,padding='same', activation='relu',name='conv5_2-relu5_2'),
              layers.Conv2D(filters=512, kernel_size=3,padding='same', activation='relu',name='conv5_3-relu5_3'),
              layers.MaxPooling2D(pool_size=2, strides=2,name='pool5'),
              layers.Conv2D(filters=4096, kernel_size=7, activation='relu',name='fc6-relu6'),
              layers.Dropout(0.5,name='do6'),
              layers.Conv2D(filters=4096, kernel_size=1, activation='relu',name='fc7-relu7'),
              layers.Dropout(0.5,name='do7'),
              layers.Conv2D(filters=num_classes, kernel_size=1,activation='softmax',name='fc8-prob'),
              layers.Flatten(name='flatten'),
          ]
        )

  return model

In [ ]:
model=build_vggface()
model.summary()
keras.utils.plot_model(model,show_shapes=True, show_layer_names=True)
# model.load_weights('vgg_face_weights.h5')

In [ ]:
checkpoint_filepath = "model.ckpt"
model_checkpoint = keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(checkpoint_filepath, 'model_{epoch:02d}.weights.h5'),
    save_weights_only=True,
    save_best_only=True,
    monitor='val_loss',
    mode='min',
)

early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    mode='min',
    patience=50,
    restore_best_weights=True
)

model.compile(
    loss = keras.losses.CategoricalCrossentropy(),
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
model.fit(
    X_train,
    y_train_onehot,
    batch_size=128,
    epochs=10,
    validation_data=(X_val, y_val_onehot),
    verbose=1,
)

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test_onehot)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
stats = ModelStatistics(save_name=f"svc_{n_videos}_signer_test_{len(y_test)}", save_dir="signer/plots")

In [ ]:
stats.print_accuracy(y_test_onehot, y_pred)

In [ ]:
stats.plot_confusion_matrix(y_test, y_pred, save=True, plot=True